# The Stanford Sentiment Treebank 
The Stanford Sentiment Treebank consists of sentences from movie reviews and human annotations of their sentiment. The task is to predict the sentiment of a given sentence. We use the two-way (positive/negative) class split, and use only sentence-level labels.

In [1]:
from IPython.display import display, Markdown
with open('../../doc/env_variables_setup.md', 'r') as fh:
    content = fh.read()
display(Markdown(content))

Environment variables that need to be defined:   
`export DIR_PROJ=your_path_git_repository`  
`export PYTHONPATH=$DIR_PROJ/src`  
`export PATH_TENSORBOARD=your_path_tensorboard`  
`export PATH_DATASETS=your_path_datasets`  
`export PROJECT_ID=your_gcp_project_id`  
`export BUCKET_NAME=your_gcp_gs_bucket_name`  
`export BUCKET_TRANSLATION_NAME=your_gcp_gs_bucket_translation_name`  
`export BUCKET_STAGING_NAME=your_gcp_gs_bucket_staging_name` 
`export REGION=your_region`  
`export PATH_SAVE_MODEL=your_path_to_save_model`  
`export CLOUDSDK_PYTHON=your_path/conda-env/env_gcp_sdk/bin/python`  
`export CLOUDSDK_GSUTIL_PYTHON=your_path/conda-env/env_gcp_sdk/bin/python`  

- Use local Jupyter Lab 
    - you need to have the `jupyter-notebook` Anaconda python environment created [link](local_jupyter_lab_installation.md) 
    - you need to have the `jupyter-notebook` Anaconda python environment activated [link](local_jupyter_lab_installation.md) 
    - then define the environment variables above (copy and paste) 
    - you need to have the `env_multilingual_class` Anaconda python environment created [link](local_jupyter_lab_installation.md)  
    - start Jupyter Lab:  `jupyter lab` 
    - open a Jupyter Lab notebook from `notebook/` 
     - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`
    - choose the proper Anaconda python environment:  `Python [conda env:env_multilingual_class]` [link](conda_env.md) 
    - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`


- Use GCP Jupyter Lab 
    - Go on GCP
    - open a Cloud Shell
    - `ssh-keygen -t rsa -b 4096 -C firstName_lastName`
    - `cp .ssh/id_rsa.pub .`
    - use Cloud Editor to edit this file `id_rsa.pub` and copy the full content
    - Go on Compute Engine -> Metadata
    - Click SSH Keys
    - Click Edit
    - Click + Add item, copy the content of `id_rsa.pub`
    - You should see firstName_lastName of the left
    - Click Save
    - you need to start a AI Platform instance 
    - open a Jupyter Lab terminal and got to `/home/gcp_user_name/`
    - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`
    - then `cd proj_multilingual_text_classification/`
    - create the Anacond Python environment `conda env create -f env/environment.yml`
    - create a file `config.sh` in `/home` with the following information: 
    ```
    #!/bin/bash
    
    echo "applying some configuration ..."
    git config --global user.email user_email
    git config --global user.name user_name
    git config --global credential.helper store
        
    # Add here the enviroment variables from above below
    # [EDIT ME]
    export DIR_PROJ=your_path_git_repository
    export PYTHONPATH=$DIR_PROJ/src
  
    cd /home/gcp_user_name/
    
    conda activate env_multilingual_class

    export PS1='\[\e[91m\]\u@:\[\e[32m\]\w\[\e[0m\]$'
    ```
    - Got to AI Platform Notebook, select your instance and click "Reset".
    - Wait and reshreh you Web browser with the Notebook


## Import Packages

In [2]:
import tensorflow as tf
import tensorflow_datasets

from tensorflow.keras.utils import to_categorical

from transformers import (
    BertConfig,
    BertTokenizer,
    XLMRobertaTokenizer,
    TFBertModel,
    TFXLMRobertaModel,
    TFBertForSequenceClassification,
    glue_convert_examples_to_features,
    glue_processors
)

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

from google.cloud import storage

import math
import numpy as np
import os
import glob
import time
from datetime import timedelta
import shutil
from datetime import datetime
import pickle
import re

## Check configuration

In [3]:
print(tf.version.GIT_VERSION, tf.version.VERSION)

v2.1.0-rc2-17-ge5bf8de 2.1.0


In [4]:
print(tf.keras.__version__)

2.2.4-tf


In [5]:
gpus = tf.config.list_physical_devices('GPU')
if len(gpus)>0:
    for gpu in gpus:
        print('Name:', gpu.name, '  Type:', gpu.device_type)
else:
    print('No GPU available !!!!')

No GPU available !!!!


## Define Paths

In [6]:
try:
    data_dir=os.environ['PATH_DATASETS']
except KeyError:
    print('missing PATH_DATASETS')
try:   
    tensorboard_dir=os.environ['PATH_TENSORBOARD']
except KeyError:
    print('missing PATH_TENSORBOARD')
try:   
    savemodel_dir=os.environ['PATH_SAVE_MODEL']
except KeyError:
    print('missing PATH_SAVE_MODEL')

## Import local packages

In [7]:
import preprocessing.preprocessing as pp
import utils.model_metrics as mm
import utils.model_utils as mu

/home/.conda-env/env_multilingual_class/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [8]:
import importlib
importlib.reload(pp);
importlib.reload(mm);
importlib.reload(mu);

## Loading a data from Tensorflow Datasets

In [9]:
data, info = tensorflow_datasets.load(name="imdb_reviews",
                            data_dir=data_dir,
                            as_supervised=True,
                            with_info=True)

INFO:absl:No config specified, defaulting to first: imdb_reviews/plain_text
INFO:absl:Load pre-computed datasetinfo (eg: splits) from bucket.
INFO:absl:Loading info from GCS for imdb_reviews/plain_text/1.0.0
INFO:absl:Generating dataset imdb_reviews (/home/vera_luechinger/tensorflow_datasets/imdb_reviews/plain_text/1.0.0)


INFO:absl:Downloading http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz into /home/vera_luechinger/tensorflow_datasets/downloads/ai.stanfor.edu_amaas_sentime_aclImdb_v1PaujRp-TxjBWz59jHXsMDm5WiexbxzaFQkEnXc3Tvo8.tar.gz.tmp.32f800b8846946229550f2f5b6ce46f7...
INFO:absl:Generating split train


Shuffling and writing examples to /home/vera_luechinger/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteV7AXDJ/imdb_reviews-train.tfrecord


INFO:absl:Done writing /home/vera_luechinger/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteV7AXDJ/imdb_reviews-train.tfrecord. Shard lengths: [25000]
INFO:absl:Generating split test


Shuffling and writing examples to /home/vera_luechinger/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteV7AXDJ/imdb_reviews-test.tfrecord


INFO:absl:Done writing /home/vera_luechinger/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteV7AXDJ/imdb_reviews-test.tfrecord. Shard lengths: [25000]
INFO:absl:Generating split unsupervised


Shuffling and writing examples to /home/vera_luechinger/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteV7AXDJ/imdb_reviews-unsupervised.tfrecord


INFO:absl:Done writing /home/vera_luechinger/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteV7AXDJ/imdb_reviews-unsupervised.tfrecord. Shard lengths: [50000]
INFO:absl:Skipping computing stats for mode ComputeStatsMode.AUTO.
INFO:absl:Constructing tf.data.Dataset for split None, from /home/vera_luechinger/tensorflow_datasets/imdb_reviews/plain_text/1.0.0


Dataset imdb_reviews downloaded and prepared to /home/vera_luechinger/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [11]:
data_valid = data['test'].take(1000)


### Checking basic info from the metadata

In [10]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [12]:
pp.print_info_dataset(info)

Labels:
      ['neg', 'pos']

Number of label:
      2

Structure of the data:
      dict_keys(['text', 'label'])

Number of entries:
   Train dataset: 25000
   Test dataset:  25000
--> validation dataset not defined


In [13]:
data

{'test': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>,
 'train': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>,
 'unsupervised': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>}

In [14]:
data.keys()

dict_keys(['test', 'train', 'unsupervised'])

In [15]:
pp.print_info_data(data['train'])

# Structure of the data:

   <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>

# Output shape of one entry:
   (TensorShape([]), TensorShape([]))

# Output types of one entry:
   (tf.string, tf.int64)

# Output typesof one entry:
   (<class 'tensorflow.python.framework.ops.Tensor'>, <class 'tensorflow.python.framework.ops.Tensor'>)
 

# Shape of the data:

   (25000, 2)
   ---> 25000 entries
   ---> 2 dim
           [text            / label           ]
           [()              / ()              ]
           [|S709           / |S1             ]


# Examples of data:
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revo

## Define parameters of the model

In [17]:
# models
#MODELS = [(TFBertModel,     BertTokenizer,       'bert-base-multilingual-uncased'),
#          (OpenAIGPTModel,  OpenAIGPTTokenizer,  'openai-gpt'),
#          (GPT2Model,       GPT2Tokenizer,       'gpt2'),
#          (CTRLModel,       CTRLTokenizer,       'ctrl'),
#          (TransfoXLModel,  TransfoXLTokenizer,  'transfo-xl-wt103'),
#          (XLNetModel,      XLNetTokenizer,      'xlnet-base-cased'),
#          (XLMModel,        XLMTokenizer,        'xlm-mlm-enfr-1024'),
#          (DistilBertModel, DistilBertTokenizer, 'distilbert-base-cased'),
#          (RobertaModel,    RobertaTokenizer,    'roberta-base'),
#          (XLMRobertaModel, XLMRobertaTokenizer, 'xlm-roberta-base'),
#         ]
MODELS = [(TFBertModel,         BertTokenizer,       'bert-base-multilingual-uncased'),
          (TFXLMRobertaModel,   XLMRobertaTokenizer, 'jplu/tf-xlm-roberta-base')]
model_index = 0 # BERT
model_class        = MODELS[model_index][0] # i.e TFBertModel
tokenizer_class    = MODELS[model_index][1] # i.e BertTokenizer
pretrained_weights = MODELS[model_index][2] #'i.e bert-base-multilingual-uncased'

# Maxium length, becarefull BERT max length is 512!
MAX_LENGTH = 128

# define parameters
BATCH_SIZE_TRAIN = 32
BATCH_SIZE_TEST = 32
BATCH_SIZE_VALID = 64
EPOCH = 2

# extract parameters
size_train_dataset  = info.splits['train'].num_examples
size_valid_dataset = np.shape(np.array(list(data_valid.as_numpy_iterator())))[0]
number_label = info.features["label"].num_classes

# computer parameter
STEP_EPOCH_TRAIN = math.ceil(size_train_dataset/BATCH_SIZE_TRAIN)
STEP_EPOCH_VALID = math.ceil(size_valid_dataset/BATCH_SIZE_VALID)


print('Dataset size:          {:6}/{:6}'.format(size_train_dataset, size_valid_dataset))
print('Batch size:            {:6}/{:6}'.format(BATCH_SIZE_TRAIN, BATCH_SIZE_VALID))
print('Step per epoch:        {:6}/{:6}'.format(STEP_EPOCH_TRAIN, STEP_EPOCH_VALID))
print('Total number of batch: {:6}/{:6}'.format(STEP_EPOCH_TRAIN*(EPOCH+1), STEP_EPOCH_VALID*(EPOCH+1)))

#TOKENIZER = 'bert-base-multilingual-uncased'

Dataset size:           25000/  1000
Batch size:                32/    64
Step per epoch:           782/    16
Total number of batch:   2346/    48


## Tokenizer and prepare data for BERT

In [18]:
# Define the checkpoint directory to store the checkpoints
pretrained_model_dir=savemodel_dir+'/pretrained_model/'+pretrained_weights
os.makedirs(pretrained_model_dir, exist_ok=True)

In [19]:
# Load tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights, cache_dir=pretrained_model_dir)

In [21]:
# recap of input dataset
print(data['train'])
print(tf.data.experimental.cardinality(data['train']))
print(tf.data.experimental.cardinality(data_valid))
# super slow since looping over all data
#print(len(list(data['train'])))

<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>
tf.Tensor(-2, shape=(), dtype=int64)
tf.Tensor(-2, shape=(), dtype=int64)


### Additional steps for the IMDb dataset specifically

#### Cleaning

In [22]:
def preprocess_reviews(reviews):
    #REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    
    #ae, oe, ue => only for GERMAN data
    #REPLACE_UMLAUT_AE = re.compile("(ae)")
    #REPLACE_UMLAUT_OE = re.compile("(oe)")
    #REPLACE_UMLAUT_UE = re.compile("(ue)")
    
    #reviews = [REPLACE_NO_SPACE.sub("", line[0].decode("utf-8").lower()) for line in np.array(list(reviews.as_numpy_iterator()))]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line[0].decode("utf-8")) for line in np.array(list(reviews.as_numpy_iterator()))]# for line in reviews]
    #reviews = [REPLACE_UMLAUT_AE.sub("ä", line[0]) for line in reviews]
    #reviews = [REPLACE_UMLAUT_OE.sub("ö", line[0]) for line in reviews]
    #reviews = [REPLACE_UMLAUT_UE.sub("ü", line[0]) for line in reviews]
    
    return reviews

In [23]:
reviews_train_clean = preprocess_reviews(data['train'])
reviews_valid_clean = preprocess_reviews(data_valid)

#### Converting Data to GLUE Format

In [24]:
labels_train = [int(line[1].decode("utf-8")) for line in np.array(list(data['train'].as_numpy_iterator()))]
labels_valid = [int(line[1].decode("utf-8")) for line in np.array(list(data_valid.as_numpy_iterator()))]

In [25]:
train_data_np = pp.convert_np_array_to_glue_format(reviews_train_clean, labels_train)
valid_data_np = pp.convert_np_array_to_glue_format(reviews_valid_clean, labels_valid, shift=len(list(train_data_np)))

In [26]:
# Prepare data for BERT
train_dataset = glue_convert_examples_to_features(train_data_np, 
                                                  tokenizer, 
                                                  max_length=MAX_LENGTH, 
                                                  task='sst-2')
valid_dataset = glue_convert_examples_to_features(valid_data_np, 
                                                  tokenizer, 
                                                  max_length=MAX_LENGTH, 
                                                  task='sst-2')

In [27]:
# adding the number of entries
if tf.version.VERSION[0:5]=='2.2.0':
    train_dataset=train_dataset.apply(tf.data.experimental.assert_cardinality(tf.data.experimental.cardinality(data['train'])))
    valid_dataset=valid_dataset.apply(tf.data.experimental.assert_cardinality(tf.data.experimental.cardinality(data_valid))) 

In [28]:
# recap of pre processing dataset
print(train_dataset)
if tf.version.VERSION[0:5]=='2.2.0':
    print(tf.data.experimental.cardinality(train_dataset))
    print(tf.data.experimental.cardinality(valid_dataset))
    # super slow since looping over all data
    #print(len(list(train_dataset)))
else:
    print(size_train_dataset)
    print(size_valid_dataset)

<FlatMapDataset shapes: ({input_ids: (None,), attention_mask: (None,), token_type_ids: (None,)}, ()), types: ({input_ids: tf.int32, attention_mask: tf.int32, token_type_ids: tf.int32}, tf.int64)>
25000
1000


## Check the final data

In [29]:
pp.print_info_data(train_dataset,print_example=False)

# Structure of the data:

   <FlatMapDataset shapes: ({input_ids: (None,), attention_mask: (None,), token_type_ids: (None,)}, ()), types: ({input_ids: tf.int32, attention_mask: tf.int32, token_type_ids: tf.int32}, tf.int64)>

# Output shape of one entry:
   ({'input_ids': TensorShape([None]), 'attention_mask': TensorShape([None]), 'token_type_ids': TensorShape([None])}, TensorShape([]))

# Output types of one entry:
   ({'input_ids': tf.int32, 'attention_mask': tf.int32, 'token_type_ids': tf.int32}, tf.int64)

# Output typesof one entry:
   ({'input_ids': <class 'tensorflow.python.framework.ops.Tensor'>, 'attention_mask': <class 'tensorflow.python.framework.ops.Tensor'>, 'token_type_ids': <class 'tensorflow.python.framework.ops.Tensor'>}, <class 'tensorflow.python.framework.ops.Tensor'>)
 

# Shape of the data:

   (25000, 2)
   ---> 25000 batches
   ---> 2 dim
        label
           shape: ()
        dict structure
           dim: 3
           [input_ids       / attention_mask  / to

In [30]:
pp.print_detail_tokeniser(train_dataset, tokenizer)

 input_ids     ---->    attention_mask    token_type_ids    modified text                 

       101     ---->           1                 1          [ C L S ]                     
     10372     ---->           1                 1          t h i s                       
     10140     ---->           1                 1          w a s                         
     10144     ---->           1                 1          a n                           
     35925     ---->           1                 1          a b s o l u t e               
     10563     ---->           1                 1          # # l y                       
     50334     ---->           1                 1          t e r r i b l e               
     13113     ---->           1                 1          m o v i e                     
       119     ---->           1                 1          .                             
     11530     ---->           1                 1          d o n                        

## Save data as TFRecord files

In [31]:
# Create directory to save TFRecord files
tfrecord_data_dir=data_dir+'/tfrecord/imdb/bert-base-multilingual-uncased'
os.makedirs(tfrecord_data_dir, exist_ok=True)

In [32]:
pp.write_tf_data_into_tfrecord(train_dataset,tfrecord_data_dir+'/train_dataset')

In [33]:
#pp.write_tf_data_into_tfrecord(test_dataset,tfrecord_data_dir+'/test_dataset')

In [34]:
pp.write_tf_data_into_tfrecord(valid_dataset,tfrecord_data_dir+'/valid_dataset')

## Read data from TFRecord files (sanity check)

In [35]:
# TFRecords encode and store data
train_files = tf.data.TFRecordDataset(tfrecord_data_dir+'/train_dataset.tfrecord')
valid_files = tf.data.TFRecordDataset(tfrecord_data_dir+'/valid_dataset.tfrecord')

In [36]:
train_dataset2 = train_files.map(pp.parse_tfrecord_glue_files)
valid_dataset2 = valid_files.map(pp.parse_tfrecord_glue_files)

In [37]:
# adding the number of entries
if tf.version.VERSION[0:5]=='2.2.0':
    train_dataset2=train_dataset2.apply(tf.data.experimental.assert_cardinality(train_dataset2.reduce(0, lambda x, _: x + 1).numpy()))
    valid_dataset2=valid_dataset2.apply(tf.data.experimental.assert_cardinality(valid_dataset2.reduce(0, lambda x, _: x + 1).numpy()))

In [38]:
if tf.version.VERSION[0:5]=='2.2.0':
    print(tf.data.experimental.cardinality(train_dataset2))
    print(tf.data.experimental.cardinality(valid_dataset2))
else:
    print(train_dataset2.reduce(0, lambda x, _: x + 1).numpy())
    print(valid_dataset2.reduce(0, lambda x, _: x + 1).numpy())

25000
1000


In [39]:
pp.print_info_data(train_dataset2,print_example=False)

# Structure of the data:

   <MapDataset shapes: ({input_ids: (None,), attention_mask: (None,), token_type_ids: (None,)}, ()), types: ({input_ids: tf.int32, attention_mask: tf.int32, token_type_ids: tf.int32}, tf.int64)>

# Output shape of one entry:
   ({'input_ids': TensorShape([None]), 'attention_mask': TensorShape([None]), 'token_type_ids': TensorShape([None])}, TensorShape([]))

# Output types of one entry:
   ({'input_ids': tf.int32, 'attention_mask': tf.int32, 'token_type_ids': tf.int32}, tf.int64)

# Output typesof one entry:
   ({'input_ids': <class 'tensorflow.python.framework.ops.Tensor'>, 'attention_mask': <class 'tensorflow.python.framework.ops.Tensor'>, 'token_type_ids': <class 'tensorflow.python.framework.ops.Tensor'>}, <class 'tensorflow.python.framework.ops.Tensor'>)
 

# Shape of the data:

   (25000, 2)
   ---> 25000 batches
   ---> 2 dim
        label
           shape: ()
        dict structure
           dim: 3
           [input_ids       / attention_mask  / token_

In [40]:
pp.print_detail_tokeniser(train_dataset2, tokenizer)

 input_ids     ---->    attention_mask    token_type_ids    modified text                 

       101     ---->           1                 1          [ C L S ]                     
     10372     ---->           1                 1          t h i s                       
     10140     ---->           1                 1          w a s                         
     10144     ---->           1                 1          a n                           
     35925     ---->           1                 1          a b s o l u t e               
     10563     ---->           1                 1          # # l y                       
     50334     ---->           1                 1          t e r r i b l e               
     13113     ---->           1                 1          m o v i e                     
       119     ---->           1                 1          .                             
     11530     ---->           1                 1          d o n                        

In [41]:
train_dataset

<FlatMapDataset shapes: ({input_ids: (None,), attention_mask: (None,), token_type_ids: (None,)}, ()), types: ({input_ids: tf.int32, attention_mask: tf.int32, token_type_ids: tf.int32}, tf.int64)>

In [42]:
train_dataset2

<MapDataset shapes: ({input_ids: (None,), attention_mask: (None,), token_type_ids: (None,)}, ()), types: ({input_ids: tf.int32, attention_mask: tf.int32, token_type_ids: tf.int32}, tf.int64)>

In [43]:
for i in train_dataset2:
    print(i)
    break

({'input_ids': <tf.Tensor: shape=(128,), dtype=int32, numpy=
array([  101, 10372, 10140, 10144, 35925, 10563, 50334, 13113,   119,
       11530,   112,   162, 10346, 30570, 10390, 10104, 10151, 15550,
       19864, 10142, 10362, 10721, 16228, 23567,   119, 11422, 10320,
       11838, 26826,   117, 10502, 10372, 14650, 24817, 10346, 10487,
       43060, 11892, 10104, 10441,   119, 12818, 10487, 11838, 22141,
       12296, 10497, 19619, 10537, 10372, 13113,   112,   161, 89215,
       21131, 34238, 10107, 71712,   119, 10372, 13113, 10127, 10144,
       11607, 16451, 14086, 10763, 26294, 17415,   119, 10103, 10889,
       26584, 89110, 21975, 10342, 12671, 10704, 10103, 13289, 10115,
       41535, 10342, 13566, 10487, 16379, 10139, 14236, 10107,   119,
       10796, 10173, 56902, 24445, 14889, 14061, 10658,   117, 10110,
       10483, 48743, 11157, 35013, 10171, 19864, 10142, 10140, 20587,
       10502,   143, 26584, 89110, 48740, 72670, 10104,   143, 13113,
       10203, 10140, 54255, 3

In [44]:
for i in train_dataset:
    print(i)
    break

({'input_ids': <tf.Tensor: shape=(128,), dtype=int32, numpy=
array([  101, 10372, 10140, 10144, 35925, 10563, 50334, 13113,   119,
       11530,   112,   162, 10346, 30570, 10390, 10104, 10151, 15550,
       19864, 10142, 10362, 10721, 16228, 23567,   119, 11422, 10320,
       11838, 26826,   117, 10502, 10372, 14650, 24817, 10346, 10487,
       43060, 11892, 10104, 10441,   119, 12818, 10487, 11838, 22141,
       12296, 10497, 19619, 10537, 10372, 13113,   112,   161, 89215,
       21131, 34238, 10107, 71712,   119, 10372, 13113, 10127, 10144,
       11607, 16451, 14086, 10763, 26294, 17415,   119, 10103, 10889,
       26584, 89110, 21975, 10342, 12671, 10704, 10103, 13289, 10115,
       41535, 10342, 13566, 10487, 16379, 10139, 14236, 10107,   119,
       10796, 10173, 56902, 24445, 14889, 14061, 10658,   117, 10110,
       10483, 48743, 11157, 35013, 10171, 19864, 10142, 10140, 20587,
       10502,   143, 26584, 89110, 48740, 72670, 10104,   143, 13113,
       10203, 10140, 54255, 3